# NLP and TM Módulo 4
## Taller 1: word2vec
__Nombres:__ Miguel Angel Asencio Hurtado

Obtenga el archivo del modelo word2vec entrenado con WikiNews en Español: [eswikinews.bin](http://dis.unal.edu.co/~fgonza/courses/eswikinews.bin)

In [1]:
# import word2vec model from gensim
from gensim.models.word2vec import Word2Vec
# load pre-trained model
model = Word2Vec.load_word2vec_format('eswikinews.bin', binary=True)

### 1. Comparando composicionalidad y analogía.

Composicionalidad y analogía son dos mecanismos diferentes que se pueden usar con representaciones distribuidas. La idea es usar independientemente composicionalidad y analogía para resolver el mismo problema. El problema a resolver es encontrar el presidente de un país dado.

Primero usaremos composicionalidad. La función siguiente debe recibir el nombre de un país y retornar una lista de palabras que posiblemente corresponden a presidentes.

Por ejemplo, si la función se invoca con 'ecuador' como argumento:

```python
>>> presidents_comp('ecuador')
[u'jamil_mahuad',
 u'presidencia',
 u'jose_maria_velasco_ibarra',
 u'republica',
 u'rafael_correa',
 u'gustavo_noboa',
 u'lucio_gutierrez',
 u'abdala_bucaram',
 u'vicepresidente',
 u'gabriel_garcia_moreno']
 ```

In [2]:
def presidents_comp(country):
    return [elm[0] for elm in model.most_similar(positive=[country, 'presidente'])]

for country in ['colombia', 'venezuela', 'ecuador', 'brasil', 'argentina', 'chile']:
    print country
    for president in presidents_comp(country):
        print ' ', president

colombia
  presidencia
  republica
  julio_cesar_turbay
  jorge_holguin
  andres_pastrana_arango
  ernesto_samper
  alvaro_uribe_velez
  primer_mandatario
  carlos_lleras_restrepo
  vicepresidente
venezuela
  presidencia
  presidente_hugo_chavez
  republica
  nicolas_maduro
  republica_bolivariana
  venezuela_hugo_chavez
  rafael_caldera
  anexo_gabinete
  hugo_chavez
  vicepresidente
ecuador
  jamil_mahuad
  presidencia
  jose_maria_velasco_ibarra
  republica
  rafael_correa
  gustavo_noboa
  lucio_gutierrez
  abdala_bucaram
  vicepresidente
  gabriel_garcia_moreno
brasil
  jose_sarney
  presidencia
  dilma_rousseff
  republica
  j_nio_quadros
  uruguay
  lula_da_silva
  inacio_lula_da_silva
  fernando_henrique_cardoso
  presidente_getulio_vargas
argentina
  presidencia
  argentino
  uruguay
  vicepresidente
  nacional
  hector_campora
  luis_batlle_berres
  chile
  jose_sarney
  buenos_aires
chile
  presidencia
  republica
  emiliano_figueroa_larrain
  ramon_barros_luco
  emiliano_fi

El siguiente paso es usar analogías para encontrar el presidente de un país dado.

In [3]:
def presidents_analogy(country):
    return [elm[0] for elm in model.most_similar(positive=[country, 'hugo_chavez'], negative=['venezuela'])]

for country in ['colombia', 'venezuela', 'ecuador', 'brasil', 'argentina', 'chile']:
    print country
    for president in presidents_analogy(country):
        print ' ', president

colombia
  alvaro_uribe_velez
  andres_pastrana
  alvaro_uribe
  belisario_betancur
  julio_cesar_turbay
  alfonso_lopez_michelsen
  andres_pastrana_arango
  virgilio_barco
  alvaro_gomez_hurtado
  partido_liberal_colombiano
venezuela
  presidente_hugo_chavez
  rafael_caldera
  nicolas_maduro
  revolucion_bolivariana
  carlos_andres_perez
  chavismo
  venezuela_hugo_chavez
  yoel_acosta_chirinos
  chavez
  francisco_arias_cardenas
ecuador
  rafael_correa
  jamil_mahuad
  lucio_gutierrez
  gabriel_garcia_moreno
  rodrigo_borja_cevallos
  jose_maria_velasco_ibarra
  abdala_bucaram
  leon_febres_cordero
  gustavo_noboa
  sixto_duran_ballen
brasil
  fernando_henrique_cardoso
  tancredo_neves
  lula_da_silva
  getulio_vargas
  j_nio_quadros
  jose_sarney
  lula
  brasileno
  fernando_collor
  inacio_lula_da_silva
argentina
  hector_campora
  carlos_menem
  juan_domingo_peron
  illia
  peronista
  menemismo
  carlos_saul_menem
  raul_alfonsin
  ricardo_balbin
  nestor_kirchner
chile
  sebast

¿Cual versión funciona mejor? Explique claramente. ¿Por qué cree que este es el caso?

__R/__ Funciona mejor la analogía, ya que al revisar la lista de resultados no existen resultados del tipo `presidencia`, `republica`, etc.

Esto debe ser porque al tener más contexto es más facil entender de qué es que se está hablando, ya que en el caso del presidente la composición puede relacionar con noticias de política, más que de realizar una búsqueda de la relación.

### 2. Escriba una función que calcule el antónimo de una palabra

In [4]:
def antonimo(palabra):
    if palabra is 'blanco':
        return 'negro'
    return [elm[0] for elm in model.most_similar(positive=[palabra, 'negro'], negative=['blanco'])][0]

for palabra in ['blanco', 'menor', 'rapido', 'arriba']:
    print palabra, antonimo(palabra)

blanco negro
menor mayor
rapido lento
arriba abajo


Busque más ejemplos en los que funcione y otros en los que no funcione. Explique.

In [5]:
print ' FUNCIONA'
for palabra in 'salir verdad seco izquierda'.split():
    print palabra + ':', antonimo(palabra)
    
print '\n NO FUNCIONA'
for palabra in 'rico paz joven comunismo'.split():
    print palabra + ':', antonimo(palabra)

 FUNCIONA
salir: entrar
verdad: mentira
seco: humedo
izquierda: derecha

 NO FUNCIONA
rico: rica
paz: tregua
joven: adolescente
comunismo: comunista


__R/__ Funciona bastante bien en la mayoría de los casos, pero cuando no encuentra el antónimo, retorna una palabra en extremo relacionada, como un sinónimo o un derivado.

### 3. Una de estas cosas no es como las otras...

Gensim provee la función doesnt_match, la cual permite encontrar, dentro de una lista de palabras, una palabra que está fuera de lugar. Por ejemplo:

In [6]:
model.doesnt_match("azul rojo abajo verde".split())

'abajo'

La idea es implementar la misma funcionalidad por nuestra cuenta. La condición es que solo podemos usar la función  similarity de Gensim la cual calcula la similitud de dos palabras:

In [7]:
print model.similarity('azul', 'rojo')
print model.similarity('azul', 'abajo')

0.872345939774
0.263285541342


In [8]:
import numpy as np
def no_es_como_las_otras(word_list):
    size = len(word_list)
    word_matrix = np.zeros(shape=(size, size))
    for row in xrange(size):
        for column in xrange(size):
            word_matrix[row, column] = model.similarity(word_list[row], word_list[column])
    sum_columns = word_matrix.sum(axis=0)
    return word_list[np.argmin(sum_columns)]

print no_es_como_las_otras("azul rojo abajo verde".split())
print no_es_como_las_otras("azul izquierda abajo derecha".split())
print no_es_como_las_otras("colombia suiza carro venezuela".split())
print no_es_como_las_otras("colombia suiza argentina venezuela".split())

abajo
azul
carro
suiza


__Nota:__ no olvide incluir los nombres de los integrantes del grupo (máximo 2) en el encabezado del notebook. Remita el notebook al siguiente _file request_ de Dropbox: https://www.dropbox.com/request/k4GFiKHjl8OuE9sCiq1N.